In [ ]:
!pip install langchain==0.3.8
!pip install langchain_community #0.3.9
!pip install tiktoken
!pip install faiss-cpu
!pip install gradio
!pip install Pillow
!pip install transformers #4.46.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 42.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.5/409.5 kB 34.1 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.19
    Uninstalling langchain-core-0.3.19:
      Successfully uninstalled langchain-core-0.3.19
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.7
    Uninstalling langchain-0.3.7:
      Successfully uninstalled langchain-0.3.7
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 74.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 44.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 62.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.2/320.2 kB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━

In [ ]:
import langchain
# print(langchain.__version__) #0.3.7
!pip show transformers #Version: 0.3.8


Name: transformers
Version: 4.46.2
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /usr/local/lib/python3.10/dist-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: peft, sentence-transformers


In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from google.colab import userdata
import os
import openai

from langchain import hub
from langchain.vectorstores import FAISS
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain.agents import create_react_agent, initialize_agent, Tool, ZeroShotAgent, load_tools, AgentExecutor, AgentType
from langchain.chat_models import ChatOpenAI
# from langchain.agents import AgentType
from langchain.memory import ConversationBufferMemory
from langchain.tools import Tool

from sentence_transformers import SentenceTransformer, util
import pandas as pd
import openai
import json
from google.colab import userdata
import os
import re
import ast
import requests
from PIL import Image
from io import BytesIO
from transformers import CLIPTokenizer
import torch

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [ ]:
# _------ OPENAI API KEY ------------
api_key = userdata.get('OA_API')
os.environ['OPENAI_API_KEY'] = api_key
openai.api_key = os.getenv('OPENAI_API_KEY')


# -----  Models used ---------------
np.random.seed(42)
llm_gpt4 = ChatOpenAI(model="gpt-4o-mini",temperature=0.3)
llm_gpt3 = ChatOpenAI(model="gpt-3.5-turbo",temperature=0)
model_transformer = SentenceTransformer('clip-ViT-B-32')
tokenizer_clip = CLIPTokenizer.from_pretrained("openai/clip-vit-base-patch32")

# ------------ Read DATA ------
# data = pd.read_csv("/content/drive/MyDrive/GenAI@IISC_Group6/DB_Amazon_cellphone_features.csv")
data = pd.read_csv("/content/Amazon_cellphone_features.csv")
data["ProductDetailsFineTuned"] = "-ASIN : " + data["parent_asin"] + "\n" + data["ProductDetailsFineTuned"]
features_list = data["ProductDetailsFineTuned"].tolist()
data.head()

,main_category,title,average_rating,rating_number,features,description,price,images,videos,store,categories,details,parent_asin,bought_together,subtitle,author,Details,ProductDetails,ProductDetailsFineTuned
0,Cell Phones & Accessories,SAMSUNG Galaxy A52A525F128GB 6GB RAMFactory Un...,4.3,685,4G Bands123457812172026283238404166.Cameras64 ...,Behold the comfortablesleek curves on Galaxy A...,199.99,[{'thumb': 'https://m.media-amazon.com/images/...,[{'title': 'Samsung Galaxy A52 - Honest Review...,SAMSUNG,"['Cell Phones & Accessories', 'Cell Phones']",{'Package Dimensions': '7.68 x 4.76 x 2.09 inc...,B098BK2SJ5,NaN,NaN,NaN,Mobile ASIN : B098BK2SJ5\nMobile Brand : samsu...,- Brand Name: Samsung\n- Mobile Name: Samsung ...,-ASIN : B098BK2SJ5\n- Brand Name: Samsung\n- M...
1,Cell Phones & Accessories,Moto G Power20223-Day BatteryUnlockedMade for ...,4.2,655,Carrier compatibilityAT&TVoLTEVerizonVoLTET Mo...,Moto G Power 2022Up to three days of battery l...,129.99,[{'thumb': 'https://m.media-amazon.com/images/...,[{'title': 'Moto G 5g Watch Before You Get It ...,Motorola,"['Cell Phones & Accessories', 'Cell Phones']",{'Product Dimensions': '6.57 x 3.03 x 0.35 inc...,B0BL9TKSBL,NaN,NaN,NaN,Mobile ASIN : B0BL9TKSBL\nMobile Brand : motor...,- Brand Name: Motorola\n- Mobile Name: Moto G ...,-ASIN : B0BL9TKSBL\n- Brand Name: Motorola\n- ...
2,Cell Phones & Accessories,Apple iPhone 14 Plus512GB(PRODUCTRedUnlocked (...,4.5,225,6.7-inch Super Retina XDR displayadvanced came...,The iPhone 14 Plus is an excellent choice if y...,909.89,[{'thumb': 'https://m.media-amazon.com/images/...,[{'title': 'iPhone 14 vs iPhone 14 Plus - Size...,Amazon Renewed,"['Cell Phones & Accessories', 'Cell Phones']","{'Package Dimensions': '8 x 5 x 3 inches', 'It...",B0BY5V82W4,NaN,NaN,NaN,Mobile ASIN : B0BY5V82W4\nMobile Brand : amazo...,- Brand Name: Amazon Renewed\n- Mobile Name: A...,-ASIN : B0BY5V82W4\n- Brand Name: Amazon Renew...
3,Cell Phones & Accessories,SAMSUNG Galaxy S23 Cell PhoneFactory Unlocked ...,4.4,64,CAPTURE THE NIGHT IN LOW LIGHTWhether youâ€™re...,Meet Galaxy S23the phone takes you out of the ...,759.99,[{'thumb': 'https://m.media-amazon.com/images/...,[{'title': 'Do this as soon as you get your S2...,SAMSUNG,"['Cell Phones & Accessories', 'Cell Phones']",{'Package Dimensions': '6.38 x 3.46 x 1.06 inc...,B0BXKWGCSW,NaN,NaN,NaN,Mobile ASIN : B0BXKWGCSW\nMobile Brand : samsu...,- Brand Name: Samsung\n- Mobile Name: Samsung ...,-ASIN : B0BXKWGCSW\n- Brand Name: Samsung\n- M...
4,Cell Phones & Accessories,SAMSUNG Galaxy S22Plus 256GB Factory Unlocked ...,4.4,136,Renewed conditionMay have few minor scuffs or ...,Meet the SAMSUNG Galaxy S22the smartphone that...,493.00,[{'thumb': 'https://m.media-amazon.com/images/...,"[{'title': 'Samsung S22 Ultra', 'url': 'https:...",Amazon Renewed,"['Cell Phones & Accessories', 'Cell Phones']","{'Product Dimensions': '3 x 0.3 x 6 inches', '...",B0B4VCQ7MB,NaN,NaN,NaN,Mobile ASIN : B0B4VCQ7MB\nMobile Brand : amazo...,- Brand Name: Amazon Renewed\n- Mobile Name: S...,-ASIN : B0B4VCQ7MB\n- Brand Name: Amazon Renew...


In [ ]:
review_data  = pd.read_csv("/content/Amazon_cellphone_reviews.csv")
review_data["review_length"] = review_data["text"].str.split().str.len()
review_data["title_length"] = review_data["title"].str.split().str.len()
review_data = review_data[review_data["review_length"]>10]
review_data["review_text"] = "- " + review_data["text"]
review_data.head()

<ipython-input-7-b33d8ac88214>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  review_data["review_text"] = "- " + review_data["text"]


,rating,title,text,images,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase,review_length,title_length,review_text
0,1,Too many ads. Can't hook up to verizon,Too many ads. Can't set lockscreen image becau...,[],B075QWLV3J,B075QWLV3J,AHATA6X6MYTC3VNBFJ3WIYVK257A,2018-04-07 17:36:18.834,0,False,87.0,8.0,- Too many ads. Can't set lockscreen image bec...
1,5,Best phone yet,I love this phone it has everything and it doe...,[],B08P54F6HB,B0B8PDLMMS,AFAMOLAPKR3M2NPTMB362T2Q5RBQ,2021-09-08 23:51:02.923,1,True,14.0,3.0,- I love this phone it has everything and it d...
3,5,LG phone,I am very happy with my new phone. I love the ...,[],B009LRNLW2,B00MU042H0,AEPLHPUQ6UTBO2PUZGM3CQ6YG5WQ,2014-02-13 13:11:36.000,0,True,30.0,2.0,- I am very happy with my new phone. I love th...
4,5,Very nice phone,The renewed phone arrived in perfect condition...,[],B082T4F34B,B09JJCXC2M,AFOMMZPFMNVCAC6VYYGQPJSRZFTQ,2021-09-19 13:25:13.275,0,True,81.0,3.0,- The renewed phone arrived in perfect conditi...
5,4,"Clean, unlocked and as described.","Love the Size, physical and memory. Not using ...",[],B07P6SWG7T,B09JJCXC2M,AFHFWRPC2P2R4FISMWNGX5E4A7TA,2020-05-19 20:45:45.585,0,True,59.0,5.0,"- Love the Size, physical and memory. Not usin..."


In [ ]:
def extract_input_features(text, model="gpt-3.5-turbo"):
    """
    Extract features using OpenAI's chat-based models.

    :param text: The input text describing a product.
    :param model: OpenAI model to use for feature extraction.
    :return: Extracted features as text.
    """
    response = openai.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are an assistant that extracts key features of products."},
            {"role": "user", "content": f"Extract and list the key features which are available from ⁠Brand Name,⁠ ⁠⁠Mobile Name, ⁠⁠Processor, ⁠Storage, ⁠RAM, ⁠Camera, ⁠⁠Price, Sim Type, ⁠⁠Battery Life, ⁠Average Rating, ⁠⁠Count of Rating and ⁠Screen Details of the following product:\n Text : {text}.\n Make sure to provide information only from Text\n If feature details are not available in Text, Provide NA only with respect to that feature."}
        ],
        max_tokens=300,
        temperature=0.3
    )
    return response.choices[0].message.content.strip()

def recommend_similar_features(input_feature_text):
    """
    Recommend products with similar features.

    :param input_feature_text: Input features as text.
    :param product_descriptions: List of product descriptions.
    :return: List of recommended products sorted by similarity.
    """

    vectorizer = TfidfVectorizer()
    feature_vectors = vectorizer.fit_transform(
        ["\n".join([v
                    for v in extract_input_features(input_feature_text).split("\n")
                    if "NA" not in v and "Not" not in v])] + features_list
        )

    # Compute cosine similarity
    similarity_scores = cosine_similarity(feature_vectors[0:1], feature_vectors[1:]).flatten()

    # Sort products by similarity
    sorted_indices = np.argsort(-similarity_scores)
    recommendations = pd.DataFrame([
        {
            "features": features_list[idx],
            "similarity_score": similarity_scores[idx]
        }
        for idx in sorted_indices
    ])
    recommendations = recommendations[recommendations["similarity_score"]>=0.1].sort_values(by="similarity_score",ascending=False).head(5)["features"].values
    return "\n\n".join(recommendations)


In [ ]:
features_tool = Tool(
    name = "RecommendSimilarFeatures",
    func = recommend_similar_features,
    description = (
        "Recommend similar products based on features. "
        "Provide input features and a list of product descriptions."
    )
)

recommendation_agent = initialize_agent(
    tools=[features_tool],
    llm= llm_gpt4,
    # agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    memory=ConversationBufferMemory(),
)

input_product = "Suggest me mobile with Snapdragon 690"

# response = recommendation_agent.run(input_product)
response = features_tool.func(input_product)
print(response)

-ASIN : B07XYKK71P
- Brand Name: OnePlus
- Mobile Name: OnePlus Nord N10 5G Global/EU Version
- Processor: Qualcomm Snapdragon 690 5G
- Storage: 128GB UFS 2.1, expandable up to 512GB
- RAM: 6GB LPDDR4X
- Camera: 
  - Rear Camera: 64MP main + 8MP ultra-wide + 2MP macro + 2MP monochrome
  - Front Camera: 16MP
- Price: $299.99
- Sim Type: GSM Unlocked (Works with T-Mobile, AT&T, Metro, etc.)
- Summary of Description: Great budget Android smartphone with 6.49-inch FHD display, 90Hz refresh rate, quad-camera setup, Snapdragon 690 processor, and 4300mAh battery with Warp Charge 30T fast charging.
- Battery Life: 4300mAh (non-removable)
- Average Rating: 4.1
- Count of Rating: 48
- Screen Details: 6.49 inches, FHD (1080x2400), 90Hz refresh rate, IPS LCD capacitive touchscreen, 405 PPI, 20:9 aspect ratio, support for sRGB, 2.5D Corning Gorilla Glass 3

-ASIN : B07XY8YFQQ
- Brand Name: OnePlus
- Mobile Name: OnePlus Nord N10 5G
- Processor: Qualcomm SM6350 Snapdragon 690 5G
- Storage: 128GB
- R

In [ ]:
# input_product = "Suggest me mobile with Snapdragon 690"


print(response)

The recommended mobile phones with Snapdragon 690 are OnePlus Nord N10 5G Global/EU Version ($299.99) and OnePlus Nord N10 5G ($249.00).


In [ ]:
def summarize_reviews(reviews_text, model="gpt-3.5-turbo"):
    """
    Summarizes all reviews for a specific product.

    :param reviews_text: Text containing all the reviews for a product.
    :param model: OpenAI model to use for summarization.
    :return: A brief summary of the reviews.
    """
    response = openai.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are a helpful assistant that summarizes reviews."},
            {"role": "user", "content": f"Summarize the following reviews for a product in key point as list and be specific with pros and cons :\n{reviews_text}"}
        ],
        max_tokens=150,
        temperature=0.3
    )
    return response.choices[0].message.content.strip()

# Wrap the function as a LangChain Tool
summarize_reviews_tool = Tool(
    name="SummarizeReviews",
    func=summarize_reviews,
    description=(
        "This tool summarizes multiple reviews for a specific product into a brief summary. "
        "Provide all the reviews as input."
    )
)


llm = ChatOpenAI(model="gpt-3.5-turbo",temperature=0.3)

summarize_reviews_agent = initialize_agent(
    tools=[summarize_reviews_tool],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    memory=ConversationBufferMemory(),
    handle_parsing_errors=True
)

def get_reviews(reviews_data, asin):
  reviews = "\n".join(
      reviews_data[reviews_data["parent_asin"]==asin].sort_values("timestamp")["review_text"])
  return reviews

In [ ]:
filtered_asins = ([text.replace("-ASIN : ", "").strip() for text in recommend_similar_features(input_product).split("\n") if "-ASIN : " in text])
for asin in filtered_asins:
  reviews = get_reviews(review_data, asin)
  print(asin)
  summary = summarize_reviews_agent.run(reviews)
  print(summary)
  print()




B07XYKK71P


> Entering new AgentExecutor chain...
I should use the SummarizeReviews tool to get a summary of multiple reviews for this phone.
Action: SummarizeReviews
Action Input: reviews_text
Observation: Review 1:
Pros: Great quality material, comfortable fit, stylish design.
Cons: Slightly expensive compared to similar products.

Review 2:
Pros: Excellent durability, easy to clean, versatile for various activities.
Cons: Limited color options available.

Review 3:
Pros: Lightweight and breathable fabric, perfect for workouts, true to size.
Cons: Some customers reported issues with stitching coming loose after a few washes.

Summary:
- Pros: High-quality material, comfortable fit, durable, easy to clean, versatile, lightweight and breathable fabric, suitable for workouts.
- Cons: Slightly expensive, limited color options, potential issues with stitching durability.
Thought:I need to provide a summary based on the reviews for the phone, not the fabric.
Action: SummarizeReviews
Actio

In [ ]:
from langchain.prompts import PromptTemplate
def create_price_agent_title(data,column1,column2,column3):
    text_splitter = CharacterTextSplitter(chunk_size=50, chunk_overlap=20,separator='\n\n')
    texts = text_splitter.create_documents(data[column1].astype(str).tolist())
    print(texts)

    embeddings = OpenAIEmbeddings()
    db = FAISS.from_documents(texts, embeddings)
    docsearch = FAISS.from_documents(texts, embeddings)
    qa = RetrievalQA.from_chain_type(llm= llm_gpt4, chain_type="stuff", retriever=docsearch.as_retriever(search_kwargs={"k":3}))
    tool = Tool(
        name=f"{column1}",
        func= qa.run,
        description=f"""Useful for when you need to answer questions about
        the {column1} column in the Amazon Cell Phones dataset.
        The price {column2} for each phone is listed next to the PhoneName={column3}
        Only fetch answer from document search the price about cell phones {column2},in the context of cell phone price
        not about other products or off-topic questions.
        Output only PhoneId with matched criteria, do not give output of unmatched Ids""",
    )
    return tool

def create_price_agent_asin(data,column1,column2,column3):
    text_splitter = CharacterTextSplitter(chunk_size=50, chunk_overlap=20,separator='\n\n')
    texts = text_splitter.create_documents(data[column1].astype(str).tolist())
    print(texts)

    embeddings = OpenAIEmbeddings()
    db = FAISS.from_documents(texts, embeddings)
    docsearch = FAISS.from_documents(texts, embeddings)
    qa = RetrievalQA.from_chain_type(llm= llm_gpt4, chain_type="stuff", retriever=docsearch.as_retriever(search_kwargs={"k":6}))
    tool = Tool(
        name=f"{column1}",
        func= qa.run,
        description=f"""Useful for when you need to answer questions about
        the {column1} column in the Amazon Cell Phones dataset.
        The price {column2} for each phone is listed next to the PhoneId={column3}
        Only fetch answer from document search the price about cell phones {column2},in the context of cell phone price
        not about other products or off-topic questions.
        Output only PhoneId with matched criteria, do not give output of unmatched Ids""",
    )
    return tool

reAct_prompt = hub.pull("hwchase17/react")

data['price_search_title'] = 'Phone=' + data['title'] + ",Price=" + data['price'].astype(str)
price_title_tool = create_price_agent_title(data,column1="price_search_title",column2="price", column3= "title")

data['price_search_asin'] = "parent_asin=" + data['parent_asin'] + ",Price=" + data['price'].astype(str)
price_asin_tool = create_price_agent_asin(data,column1="price_search_asin",column2="price",column3="parent_asin")

# Create the agent
price_agent = create_react_agent(
    llm= llm_gpt4,
    tools=[price_title_tool, price_asin_tool],
    prompt= reAct_prompt, # PromptTemplate.from_template(new_template),
    stop_sequence=True,
)

# Create the agent executor
price_agent_executor = AgentExecutor.from_agent_and_tools(
    agent = price_agent,
    tools = [price_title_tool, price_asin_tool],
    verbose = True,
)

# user_query = "parent_asin B07XY8YFQQ, B07XYKK71P filter products with price under 250$  "
user_query = "suggest phones with price under 250$. Suggest 5 phones "
response = price_agent_executor.invoke({"input": user_query})
print(response["output"])

/usr/local/lib/python3.10/dist-packages/langsmith/client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


[Document(metadata={}, page_content='Phone=SAMSUNG Galaxy A52A525F128GB 6GB RAMFactory Unlocked (GSM ONLYNot Compatible with Verizon/Sprint/BoostInternational Model (Awesome Purple),Price=199.99'), Document(metadata={}, page_content='Phone=Moto G Power20223-Day BatteryUnlockedMade for US by Motorola4/64GB50 MP CameraDark Grove,Price=129.99'), Document(metadata={}, page_content='Phone=Apple iPhone 14 Plus512GB(PRODUCTRedUnlocked (Renewed),Price=909.89'), Document(metadata={}, page_content='Phone=SAMSUNG Galaxy S23 Cell PhoneFactory Unlocked Android Smartphone256GB Storage50MP CameraNight ModeLong Battery LifeAdaptive DisplayUS Version2023Phantom Black,Price=759.99'), Document(metadata={}, page_content='Phone=SAMSUNG Galaxy S22Plus 256GB Factory Unlocked SM-S906U1 Pink Gold (Renewed),Price=493.0'), Document(metadata={}, page_content='Phone=ZTE Cymbal Z-320 Flip Phone UNLOCKED (T-Mobile),Price=89.0'), Document(metadata={}, page_content='Phone=Apple iPod Touch 16GB (5th GenerationSpace Gra

<ipython-input-16-3279ce391197>:7: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()


[Document(metadata={}, page_content='parent_asin=B098BK2SJ5,Price=199.99'), Document(metadata={}, page_content='parent_asin=B0BL9TKSBL,Price=129.99'), Document(metadata={}, page_content='parent_asin=B0BY5V82W4,Price=909.89'), Document(metadata={}, page_content='parent_asin=B0BXKWGCSW,Price=759.99'), Document(metadata={}, page_content='parent_asin=B0B4VCQ7MB,Price=493.0'), Document(metadata={}, page_content='parent_asin=B06WGL82S6,Price=89.0'), Document(metadata={}, page_content='parent_asin=B00VQQ294C,Price=205.0'), Document(metadata={}, page_content='parent_asin=B07L6QY1TJ,Price=94.99'), Document(metadata={}, page_content='parent_asin=B09JFJ1Q5C,Price=678.39'), Document(metadata={}, page_content='parent_asin=B09X9FC88M,Price=769.0'), Document(metadata={}, page_content='parent_asin=B08DXKVYQK,Price=129.99'), Document(metadata={}, page_content='parent_asin=B0748NBWDP,Price=109.99'), Document(metadata={}, page_content='parent_asin=B0BGYC7HYQ,Price=849.0'), Document(metadata={}, page_cont

In [ ]:
# ------------ Keeping some code safe -----------

# price_agent = initialize_agent(
#     tools=[price_title_tool, price_asin_tool],
#     llm= llm_gpt4,
#     agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
#     verbose=True,
#     memory=ConversationBufferMemory(),  # Add memory for context
# )

# # Create the agent
# price_agent_title = create_react_agent(
#     llm= llm_gpt4,
#     tools=[price_title_tool],
#     prompt=new_temlate,
#     stop_sequence=True,
# )



# # Create the agent
# price_agent_asin = create_react_agent(
#     llm= llm_gpt4,
#     tools=[price_asin],
#     prompt=reAct_prompt,
#     stop_sequence=True,
# )

# # Create the agent executor
# price_agent_executor_asin = AgentExecutor.from_agent_and_tools(
#     agent = price_agent_asin,
#     tools = [price_asin_tool],
#     verbose = True,
# )
# new_template = """Answer the following questions as best you can. You have access to the following tools:

# {tools}
# Use price_asin_tool when you find word "ASIN" in {input}

# Use the following format:

# Question: the input question you must answer
# Thought: you should always think about what to do
# Action: the action to take, should be one of [{tool_names}]
# Action Input: the input to the action
# Observation: the result of the action
# ... (this Thought/Action/Action Input/Observation can repeat N times)
# Thought: I now know the final answer
# Final Answer: the final answer to the original input question

# Begin!

# Question: {input}
# Thought:{agent_scratchpad}"""

# print(new_template)

In [ ]:
user_query = "price phoneId=B07XY8YFQQ, B07XYKK71P "
response = price_agent_executor.invoke({"input": user_query})
print(response["output"])



> Entering new AgentExecutor chain...
I need to find the prices for the specified phone IDs. I'll start by checking the price for the first phone ID, B07XY8YFQQ.

Action: price_search_asin  
Action Input: B07XY8YFQQ  The price for the parent ASIN B07XY8YFQQ is $249.00.I have found the price for the first phone ID. Now, I will check the price for the second phone ID, B07XYKK71P.

Action: price_search_asin  
Action Input: B07XYKK71P  The price for the item with parent ASIN B07XYKK71P is $299.99.I now know the final answer.  
Final Answer: The prices are $249.00 for phoneId B07XY8YFQQ and $299.99 for phoneId B07XYKK71P.

> Finished chain.
The prices are $249.00 for phoneId B07XY8YFQQ and $299.99 for phoneId B07XYKK71P.


In [ ]:
def routing_agent(user_query):
    routing_prompt = (
        "You are an intelligent assistant that routes user queries to specific agents. "

        "Based on the requested features in user query determine which agent should handle it:\n"
        f"User Query: {user_query}\n"

        " Include features_tool if features {Brand Name, RAM, Storage, Camera, Price, Sim Type, Battery Life, Screen Details, Processor} found :\n"  # determine which agent should handle it
        " Also include price_tool if Price or dollar or $ details found, ; "

        "Available agents are:\n"
        "1. features_tool\n"
        "2. price_tool\n"
        "3. ImageSearchTool\n"
        "Respond with all the names of appropriate agents in python list."
        # , and features found in a python dictionary like {features_tool: 12GB RAM} if RAM found in inout query; {price: 200$} if price found in input query ."
    )
        # Modified section to fix the issue
    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": routing_prompt}],
    )
    return response.choices[0].message.content.strip()

def filter_price_products_byAsin(product_asins, max_price, min_price):
  filtered_data = data[(data["parent_asin"].isin(product_asins)) & (data["price"] < max_price) & (data["price"] > min_price)]
  return filtered_data

import re

def remove_price_line(text):
  # Use a regular expression to find and remove lines containing "Price:"
  # pattern = r"- Price:.*\n"  # Matches lines starting with "- Price:" and ending with a newline
  # cleaned_text = re.sub(pattern, "", text, flags=re.MULTILINE)
  cleaned_text = "\n".join([txt for txt in text.split("\n") if "Price" not in txt])
  return cleaned_text

# Main function for handling user input and routing
def orchestrate_agents(user_query, prompt_features):
    agent_names = route_query_to_agent(user_query).strip()
    filtered_products = []
    agent_output = []
    reviews_summary = {}

    for agent_name in ast.literal_eval(agent_names):
        print("executing Agent " + agent_name)
        if agent_name == "features_tool":
          print("Running Feature_agent......")

              [
                  text.replace("-ASIN : ", "").strip()
                  for text in recommend_similar_features(cleaned_prompt_features).split("\n")
                  i cleaned_prompt_features = remove_price_line(prompt_features)
          agent_output = recommendation_agent.run({"input": cleaned_prompt_features})
          print("Clean Prompt : ", cleaned_prompt_features)
          filtered_asins = (f "-ASIN : " in text
                  ]
              )

          print("Filtered ASIN : ", filtered_asins)
          print("Running Reviews Agent")

          for asin in filtered_asins:
            reviews = get_reviews(review_data, asin)
            summary = summarize_reviews_agent.run(reviews)
            reviews_summary[asin] = summary


        elif agent_name == "price_tool":
          finetuned_user_query = user_query
          if agent_output:
            filtered_price_prompt = ([text.replace("- ", "").strip() for text in prompt_features.split("\n") if "Price: " in text])
            filtered_asins = ([text.replace("-ASIN : ", "").strip() for text in recommend_similar_features(user_query).split("\n") if "-ASIN : " in text])
            finetuned_user_query = "Suggest products with parent_asin {0} and price {1}".format(filtered_asins, filtered_price_prompt)
            print(finetuned_user_query)

          print("Running Price_agent......")
          price_agent_executor.invoke({"input": finetuned_user_query})

        # elif agent_name == "ImageSearchTool":
            # image_agent_output = image_agent({"input": user_query})
        else:
            return "I'm sorry, I didn't understand your query."

        return reviews_summary

input_user_prompt = "Suggest me mobile 6GB RAM range from 250 to 350$"
prompt_features = "\n".join([v
                    for v in extract_input_features(input_user_prompt).split("\n")
                    if "NA" not in v and "Not" not in v])
print(prompt_features)
reviews_summary = orchestrate_agents(input_user_prompt, prompt_features)

- RAM: 6GB
- Price: $250 to $350
executing Agent features_tool
Running Feature_agent......


> Entering new AgentExecutor chain...
I need to find products that have similar features to those with 6GB of RAM. 
Action: RecommendSimilarFeatures
Action Input: "RAM: 6GB"
Observation: -ASIN : B07XY8YFQQ
- Brand Name: OnePlus
- Mobile Name: OnePlus Nord N10 5G
- Processor: Qualcomm SM6350 Snapdragon 690 5G
- Storage: 128GB
- RAM: 6GB
- Camera: 64MP (wide), 8MP (ultrawide), 2MP (depth), 2MP (macro), 16MP (front)
- Price: $249.0
- Sim Type: Nano-SIM (Dual-SIM)
- Summary of Description: Dual-sim 128GB ROM/6GB RAM 5G smartphone with immersive stereo sound, IPS LCD 90Hz display, Android 10 OxygenOS 10.5, and network compatibility for various bands.
- Average Rating: 4.0
- Count of Rating: 231
- Screen Details: 6.49-inch IPS LCD display with 1080 x 2400 pixels resolution and Corning Gorilla Glass 3 protection.

-ASIN : B098BK2SJ5
- Brand Name: Samsung
- Mobile Name: Samsung Galaxy A52 A525F 128GB 6

In [ ]:
reviews_summary

{'B07XY8YFQQ': 'The tool needs all the reviews provided to summarize them.',
 'B098BK2SJ5': 'The Samsung A52 has several positive aspects such as fast and lovely camera, easy setup process, good quality mid-priced phone, excellent camera quality, and good value for the price. However, there are some drawbacks like fragile front glass, slow fingerprint reader, bloatware installed without permission, and language issues upon arrival.',
 'B09X5QKT1C': 'The XYZ Wireless Earbuds have great sound quality, comfortable fit, sleek design, noise cancellation feature, lightweight and portable, affordable price, and come with a charging case. The only downside is that the battery life could be longer.',
 'B09SZRGR28': 'The Samsung Galaxy A51 5G T-Mobile phone has received mixed reviews, with some praising its quality, comfort, and style, while others mention issues with price, color options, and warmth in colder weather.',
 'B09VB698H9': 'The phone is generally well-received for its quality, affor

In [ ]:
data

In [ ]:
input_feature_text = "Suggest me mobile 128GB RAM under 200$"
prompt_features = "\n".join([v
                    for v in extract_input_features(input_feature_text).split("\n")
                    if "NA" not in v and "Not" not in v])
print(prompt_features)
agent_names = route_query_to_agent(user_query).strip()
agent_names

In [ ]:
# SENTENCE TRANSFORMER

# ------- Preprocessing Image URLS ---------
def extract_image_data(image_data):
  image_ls = (image_data[1:-1])
  large = ""
  thumbs = ""
  try:
    image_list = ast.literal_eval(image_ls)
    filtered_image_list = [item for item in image_list if item] #remove empty dict
    large = [item.get('large') for item in filtered_image_list if item.get('large')]
    if not large:
      print("empty large")
      thumbs = [item.get('thumb') for item in filtered_image_list if item.get('thumb')]
  except (SyntaxError, ValueError, AttributeError) as e:
    return ""

  return str(large) if large else str(thumbs) #return thumbs if large is empty

def extract_image_urls(image_string):
    try:
        image_urls = ""
        image_list = ast.literal_eval(image_string)
        if isinstance(image_list, list):
            for image_url in image_list:
                return image_url
                # image_urls = image_urls + image_url + ", "
            # return image_urls
        else:
            return ""
    except (SyntaxError, ValueError):
        print("Invalid image string format")
        return ""

# ------- Preprocessing Image URLS ---------

data['feature_extract'] = data['ProductDetails'].astype(str) + ", Price:" + data['price'].astype(str) + ", Average Rating:" + data['average_rating'].astype(str)
data['image_filtered'] = data['images'].apply(extract_image_data)
data['image_urls'] = data['image_filtered'].apply(extract_image_urls)
data = data[data['image_filtered'] != ""]
data = data[data['image_urls'] != ""]

product_data= data.reset_index(drop=True).to_dict(orient='index')

In [ ]:
# ---------- TEXT IMAGE EMBEDDINGS ---------------

# Function to download and open an image from a URL
def get_image_from_url(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        image = Image.open(BytesIO(response.content))
        return image
    except (requests.exceptions.RequestException, IOError):
        return None  # Or handle the error as needed

def generate_image_embeddings(images):
  im_embeddings = []
  for image_url in images:
      image = get_image_from_url(image_url)
      if image:
          # Convert the NumPy array to a PyTorch tensor for model capability
          embeddings = model_transformer.encode(image)
          if isinstance(embeddings, (list, tuple)): # handles multi-image case
              im_embeddings.extend([torch.tensor(emb) for emb in embeddings])
          else:
              im_embeddings.append(torch.tensor(embeddings))
      else:
          print(f"Could not load image from URL: {image_url}")
  return im_embeddings

def truncate_text(text, max_length=74):
    tokens = tokenizer_clip.tokenize(text)
    if len(tokens) > max_length:
        tokens = tokens[:max_length - 1] + [tokenizer_clip.eos_token]  # Add EOS token if truncated
    return tokenizer_clip.convert_tokens_to_string(tokens)

def generate_text_embeddings(feature_data):
  feature_data = feature_data.apply(truncate_text) #VIT limitation
  text_embeddings = model_transformer.encode(feature_data.tolist())
  return text_embeddings

image_data = data["image_urls"]
feature_data = data['feature_extract']

image_embeddings = generate_image_embeddings(image_data)
text_embeddings = generate_text_embeddings(feature_data)

In [ ]:
def search_feature_text(query,embeddings):
  query_embedding = model_transformer.encode(query)
  results = util.semantic_search(query_embedding, embeddings, top_k=15)
  return results

def retreive_match_images(results, image_data):
  img_list = []
  images_filtered = []
  for result in results:
      try:
        pid=result['corpus_id']
        img_list.append(image_data[pid])
      except:
        img_list.append("")
  print("Image Links retrieved: " + str(img_list))
  for url in img_list:
    if(url):
      response = requests.get(url)
      response.raise_for_status()
      image = Image.open(BytesIO(response.content))
      images_filtered.append(image)
  return images_filtered

def search_feature_image(query, image_embeddings):
  query_embedding = model_transformer.encode(query)
  results = util.semantic_search(query_embedding, image_embeddings, top_k=15)
  return results


user_query = " 128gb cellphones"
text_results = search_feature_text(user_query, text_embeddings)
print("Matched Feature results: " + str(text_results))



for result in text_results[0]:
    pid = result['corpus_id']


In [ ]:
# image_dt = retreive_match_images(text_results[0], image_data)
# [display(image) for image in image_dt]

query ="black cellphones"
image_results = search_feature_image(query, image_embeddings)
# print("Matched Image results: " + str(image_results))
image_dt = retreive_match_images(image_results[0], image_data)
[display(image) for image in image_dt]

In [ ]:
# prompt: find common corpus_id from tx_results, im_results

common_corpus_ids = []
tx_corpus_ids = [result['corpus_id'] for result in text_results[0]]
im_corpus_ids = [result['corpus_id'] for result in image_results[0]]

for corpus_id in tx_corpus_ids:
    if corpus_id in im_corpus_ids:
        common_corpus_ids.append(corpus_id)

print("Common Corpus IDs:", common_corpus_ids)
print("Text Corpus IDs ", tx_corpus_ids)
print("Image Corpus IDs ", im_corpus_ids)

In [ ]:
# !pip install langchain
# !pip install langchain_community
from dotenv import load_dotenv,find_dotenv
load_dotenv(find_dotenv())

from langchain.chat_models import ChatOpenAI
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
from langchain.docstore.document import  Document


def create_docs(results):
    docs = []
    for result in results[0]:
        pid = result['corpus_id']
        score = result['score']
        result_string = ''
        result_string += "feature_extract:" + str(product_data[pid]['feature_extract']) +  ';' + \
                          "Parent ASIN:" + str(product_data[pid]['parent_asin']) +  ';' + \
                          "Image URLS:" + str(product_data[pid]['image_urls'])
        doc = Document(page_content=result_string)
        doc.metadata['pid'] = str(pid)
        doc.metadata['score'] = score
        docs.append(doc)
    return docs

def recommend_product_features(query):
  results= search_feature_text(query, text_embeddings)
  docs = create_docs(results)
  response=chain({"input_documents": docs, "question": query},return_only_outputs=True)
  return response['output_text']


template = """
You are a E-commerce shopping assistant for cellphone products that wants to convert customers based on the information given.

Provide details like RAM, Storage, Camera, Processor, Screen Size, Parent ASIN information also narrate pros and cons and at the end Recommend why is it good recommendedation given context in your interaction with the customer.

Use a bullet list when describing each product.

Context: {context}

User question:{question}


Your response:"""
prompt = PromptTemplate.from_template(template)

chain = load_qa_chain(ChatOpenAI(model_name= "gpt-3.5-turbo", temperature=0.2), chain_type="stuff", prompt=prompt)

In [ ]:
query="Provide good phones in black color with 512GB storage"
print(recommend_product_features(query))

In [ ]:
data[data['ProductDetails'].str.contains('Apple iPhone 14 Plus')]

In [ ]:

query=" black color"
results = search_feature_image(user_query, image_embeddings)
print("Matched results: " + str(results))
def recommend_image_features(query):
  results= search_feature_text(query, image_embeddings)
  docs = create_docs(results)
  response=chain({"input_documents": docs, "question": query},return_only_outputs=True)
  return response['output_text']

# print(recommend_image_features(query))

In [ ]:
feature_transformer_tool = Tool(
    name = "sentenceTransformerTool",
    func = recommend_product_features,
    description = (
        "Recommend similar products based on features. "
        "Provide input features and a list of product descriptions."
    )
)

image_transformer_tool = Tool(
    name = "sentenceTransformerTool",
    func = recommend_image_features,
    description = (
        "Recommend products based on image color. "
        "Provide input features with color information."
    )
)

recommendation_transformer_agent = initialize_agent(
    tools= [feature_transformer_tool],
    llm= llm_gpt4,
    # agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    memory=ConversationBufferMemory(),
)

image_transformer_agent = initialize_agent(
    tools= [image_transformer_tool],
    llm= llm_gpt4,
    # agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    memory=ConversationBufferMemory(),
)

# input_product = "Suggest me mobile with Snapdragon 690"
query="Provide good phones in black color with 512GB storage"
response = recommendation_transformer_agent.run(query)
print(response)

In [ ]:
# prompt: generate gradio interface with input text for query and text for output should be multiple labels to display product info and to display pros and cons in different section. display Pros with tick mark
!pip install gradio
import gradio as gr

def predict(query):
    # Replace this with your actual logic to process the query
    # and generate product info, pros, and cons.
    product_info = "Product Information:\n- Feature 1: Value 1\n- Feature 2: Value 2\n- Feature 3: Value 3"
    pros = "Pros:\n ✅ Pro 1\n✅ Pro 2\n✔ Pro 3"
    cons = "Cons:\n- Con 1\n- Con 2"
    return product_info, pros, cons

iface = gr.Interface(
    fn=predict,
    inputs=gr.Textbox(lines=2, placeholder="Enter your query here..."),
    outputs=[
        gr.Textbox(label="Product Information"),
        gr.Textbox(label="Pros"),
        gr.Textbox(label="Cons")
    ],
    title="Product Recommendation",
    description="Enter a query to get product recommendations, including pros and cons."
)

iface.launch()

In [ ]:
# prompt: add image output from [display(image) for image in image_dt] in gradio interface designed above. i have more than 1 product, how to loop each product and display pros, cons, images of the each product

import gradio as gr
# ... (your existing code) ...

def predict(query):
    product_info_list = []
    pros_list = []
    cons_list = []
    image_list = []

    # Assuming common_corpus_ids holds the IDs of products matching both text and image queries
    for corpus_id in common_corpus_ids:
        product_data_item = product_data.get(corpus_id)
        if product_data_item:
            product_info = f"Product Information:\n{product_data_item['feature_extract']}"
            product_info_list.append(product_info)

            # Use your LLM to generate pros and cons (replace this with your actual logic)
            pros = f"Pros:\n- Pro 1 for {product_data_item['title']}\n- Pro 2" # Example pros
            pros_list.append(pros)
            cons = f"Cons:\n- Con 1 for {product_data_item['title']}\n- Con 2"  # Example cons
            cons_list.append(cons)

            # Retrieve and display the image
            image_url = product_data_item['image_urls']
            if image_url:
                image = get_image_from_url(image_url)
                if image:
                    image_list.append(image)
                else:
                    image_list.append(None)  # Handle cases where image loading fails
            else:
                image_list.append(None)
        else:
            # Handle cases where product data is missing for a given corpus_id
            product_info_list.append("Product information not found.")
            pros_list.append("No pros found.")
            cons_list.append("No cons found.")
            image_list.append(None)


    return product_info_list, pros_list, cons_list, image_list

iface = gr.Interface(
    fn=predict,
    inputs=gr.Textbox(lines=2, placeholder="Enter your query here..."),
    outputs=[
        gr.Gallery(label="Product Information"),  # Changed to Gallery
        gr.Gallery(label="Pros"),  # Changed to Gallery
        gr.Gallery(label="Cons"),   # Changed to Gallery
        gr.Gallery(label="Product Images")  # Added Gallery for Images
    ],
    title="Product Recommendation",
    description="Enter a query to get product recommendations, including pros, cons, and images."
)

iface.launch()

In [ ]:
predict('128gb')

In [ ]:
# ----------------- Additional Testing or already Tested on --------------------------------
# input_product = 'Recommed me Samsung latest mobile phone'
# input_product = "A samsung mobile with 6.5` screen"
# input_product = "samsung galaxy a51"
# input_product = "4GB RAM and 64GB storage smartphone "
# input_product = '6.5" HD display with 90 Hz'
# input_product = 'I want new Iphone with 512GB Storage and screen size more than 6 inch'
# input_product = 'I want new samsung galaxy S22 ultra'
# input_product = 'Recommed me products with 4GB RAM  4000 Battery and Average Rating more than 4'
# input_product = 'Recommed me products with 6GB RAM and 512GB'
# input_product = 'Recommed me products with 8MP Camera and 6GB RAM'
# input_product = 'Recommed me 16GB Memory'
# input_product = 'Best mobile with 6GB Ram'
input_product = "Suggest me mobile with Snapdragon 690"

response = recommendation_agent.run(input_product)
print(response)